In [3]:
import pathlib
import delta.config as cfg
from delta.utilities import xpreader
from delta.pipeline import Pipeline
import tensorflow as tf

In [4]:

dev = tf.config.list_physical_devices()
print(dev)

def to_str(posixpath):
    return str(posixpath.resolve())   

[PhysicalDevice(name='/physical_device:CPU:0', device_type='CPU'), PhysicalDevice(name='/physical_device:GPU:0', device_type='GPU')]


In [13]:

#set paths
root = pathlib.Path('/Users/simonvanvliet/switchdrive/Biozentrum/Andreas/')
data_dir = pathlib.Path('/Users/simonvanvliet/switchdrive/Biozentrum/Andreas/rawdatalin/')

#create output dir
output_root = root / 'processed_datalin'
(output_root).mkdir(exist_ok=True) #create output data folder,  each position will be placed in a subfolder

#get config file
config_file = root / 'config_2D_caulobacter.json'
cfg.load_config(config_file)
cfg.model_file_seg = to_str(root / 'models' / 'unet_caulobacter2d_seg.hdf5') 
cfg.model_file_track = to_str(root / 'models' / 'unet_pads_track.hdf5') 

cfg.save_format = ('pickle','movie')

#get images in subfolder
movie_names = [f.name for f in sorted((data_dir).glob('*.tif*'))]

#create output subfolder
output_path = output_root
(output_path).mkdir(exist_ok=True) #create output data folder,  each position will be placed in a subfolder


Loading configuration from: /Users/simonvanvliet/switchdrive/Biozentrum/Andreas/config_2D_caulobacter.json


In [14]:
print(movie_names)

['20220311_AKS1088-1093_dense_TL01_07_R3D-1-1.tif', '20220311_AKS1088-1093_dense_TL01_07_R3D-1-2.tif', '20220311_AKS1088-1093_dense_TL01_23_R3D-1-1.tif', '20220311_AKS1088-1093_dense_TL01_27_R3D-1-1.tif', '20220311_AKS1088-1093_dense_TL01_28_R3D-1-1.tif', '20220311_AKS1088-1093_dense_TL01_29_R3D-1-1.tif']


In [15]:

for movie in movie_names:        
    #path to current position
    movie_dir = data_dir / movie
    
    #make nickname of movei (adapt to file name structure, here we take the part starting at TL and stopping before __R3D)
    start = movie.find('TL')
    end = movie.find('_R3D', start)
    movie_name_short = movie[start:end]
    
    #make subfolder for current position
    output_dir = output_path / movie_name_short
    (output_dir).mkdir(exist_ok=True)

    print('starting with movie %s' %(movie_name_short)) 
    # Init reader (use bioformats=True if working with nd2, czi, ome-tiff etc):
    im_reader = xpreader(movie_dir, use_bioformats=True)

    # Print experiment parameters to make sure it initialized properly:
    print("""Initialized experiment reader:
        - %d positions
        - %d imaging channels
        - %d timepoints"""%(im_reader.positions, im_reader.channels, im_reader.timepoints)
    )

    # Init pipeline:
    xp = Pipeline(im_reader, resfolder=to_str(output_dir))   

    # Run pipeline
    xp.process()
    
   



starting with movie TL01_07
13:30:18.040 [Thread-1] DEBUG loci.formats.ClassList - Could not find loci.formats.in.URLReader
java.lang.ClassNotFoundException: loci.formats.in.URLReader
	at java.base/jdk.internal.loader.BuiltinClassLoader.loadClass(BuiltinClassLoader.java:641) ~[na:na]
	at java.base/jdk.internal.loader.ClassLoaders$AppClassLoader.loadClass(ClassLoaders.java:188) ~[na:na]
	at java.base/java.lang.ClassLoader.loadClass(ClassLoader.java:521) ~[na:na]
	at java.base/java.lang.Class.forName0(Native Method) ~[na:na]
	at java.base/java.lang.Class.forName(Class.java:383) ~[na:na]
	at java.base/java.lang.Class.forName(Class.java:376) ~[na:na]
	at loci.formats.ClassList.parseLine(ClassList.java:196) ~[bioformats_package.jar:6.6.0]
	at loci.formats.ClassList.parseFile(ClassList.java:258) ~[bioformats_package.jar:6.6.0]
	at loci.formats.ClassList.<init>(ClassList.java:138) ~[bioformats_package.jar:6.6.0]
13:30:18.043 [Thread-1] DEBUG loci.formats.ClassList - Could not find loci.format

KeyboardInterrupt: 